# 1、Memory模块的设计思路
1. 层次1(最直接的方式)：保留一个聊天消息列表
2. 层次2(简单的新思路)：只返回最近交互的k条消息
3. 层次3(稍微复杂一点)：返回过去k条消息的简洁摘要
4. 层次4(更复杂)：从存储的消息中提取实体，并且仅返回有关当前运行中引用的实体的信息

# 2、ChatMessageHistory（最底层）
`ChatMessageHistory` 是一个用于 存储和管理对话消息 的基础类，它直接操作消息对象（如HumanMessage, AIMessage 等），是其它记忆组件的底层存储工具。
在API文档中，ChatMessageHistory 还有一个别名类：InMemoryChatMessageHistory；导包时，需使用 `from langchain.memory import ChatMessageHistory`
特点：
* 仅仅只做消息的存储，对消息不做任何其他处理（如生成摘要、缓冲、窗口等等）
* 不涉及消息的格式化，比如将消息转换为文本字符串

## 场景1：`ChatMessageHistory`组件的使用

In [1]:
#1.导入相关包
from langchain_classic.memory import ChatMessageHistory

#2.实例化ChatMessageHistory对象
history = ChatMessageHistory()
# 3.添加UserMessage
history.add_user_message("hi!")
# 4.添加AIMessage
history.add_ai_message("whats up?")
# 5.返回存储的所有消息列表
print(history.messages)

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up?', additional_kwargs={}, response_metadata={})]


## 场景2：对接LLM

In [5]:

from langchain_classic.memory import ChatMessageHistory
from langchain_ollama import ChatOllama

history = ChatMessageHistory()
history.add_ai_message("我是一个无所不能的小智")
history.add_user_message("你好，我叫小明，请介绍一下你自己")
history.add_user_message("我是谁呢？")
# 创建大模型实例
llm = ChatOllama(model='deepseek-r1:7b', base_url="http://localhost:11434")  # 使用本地大模型
response = llm.invoke(history.messages)
print(response.content)

你好！我是小智，一个由深度求索（DeepSeek）公司开发的智能助手。我是一个通用人工智能工具，能够理解和回答各种问题、提供信息、进行对话交流，并帮助用户完成多种任务。

作为一个AI，我的主要功能包括：
1. **信息检索**：我可以帮助你查找资料、解答问题。
2. **学习与推理**：虽然我没有意识和自我，但我可以通过大数据分析和算法推理来帮助你解决问题。
3. **语言支持**：我能用中文进行自然的交流，并理解并回应你的指令。

我的目标是为你提供便捷的帮助，无论是学习、工作还是生活中的问题，我都会尽力提供准确、有用的信息或解决方案。如果你有任何问题或需要帮助的地方，请随时告诉我！😊


# ConversationBufferMemory
`ConversationBufferMemory` 是一个基础的 对话记忆（Memory）组件 ，专门用于按原始顺序存储完整的对话历史。
适用场景：对话轮次较少、依赖**完整上下文**的场景（如简单的聊天机器）
特点：
* 存储完整的对话历史，不进行任何处理
* 与 Chains/Models 无缝集成
* 支持两种返回格式（通过 `return_messages` 参数控制输出格式，默认为False），return_messages=True时返回消息对象列表；return_messages=False时返回纯文本字符串

## 场景1：入门使用
举例1：

In [3]:
# 1.导入相关包
from langchain_classic.memory import ConversationBufferMemory

# 2.实例化ConversationBufferMemory对象
memory = ConversationBufferMemory()
# 3.保存消息到内存中
memory.save_context(inputs={"input": "你好，我是人类"}, outputs={"output": "你好，我是AI助手"})
memory.save_context(inputs={"input": "很开心认识你"},
                    outputs={"output": "我也是"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': 'Human: 你好，我是人类hh\nAI: 你好，我是AI助手\nHuman: 很开心认识你\nAI: 我也是'}


> !说明
> * `save_context()`方法的`inputs`参数对应地就是human消息，`outputs`参数对应地就是ai消息。因此，inputs和outputs参数对应字典值的key不一定就需要时input or output，可以是任何其他值的key，比如"input1"都是ok的
> * `load_memory_variables()`方法返回的是一个字典，key默认是“history”，这个后面在后面有大用。

举例2：

In [4]:
# 1.导入相关包
from langchain_classic.memory import ConversationBufferMemory

# 2.实例化ConversationBufferMemory对象
memory = ConversationBufferMemory(return_messages=True)
# 3.保存消息到内存中
memory.save_context({"input": "hi"}, {"output": "whats up"})
# 4.读取内存中消息（返回消息）
print(memory.load_memory_variables({}))
# 5.读取内存中消息( 访问原始消息列表)
print(memory.chat_memory.messages)

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up', additional_kwargs={}, response_metadata={})]}
[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up', additional_kwargs={}, response_metadata={})]


## 场景2：结合chain
举例1：使用PromptTemplate和默认的memory_key

In [7]:
from langchain_classic.chains.llm import LLMChain
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

# 初始化大模型
llm = ChatOllama(model='deepseek-r1:7b', base_url="http://localhost:11434")
# 创建提示
# 有两个输入键：实际输入与来自记忆类的输入 需确保PromptTemplate和ConversationBufferMemory中的键匹配
template = """你可以与人类对话。
当前对话历史: {history}
人类问题: {question}
回复:
"""
prompt = PromptTemplate.from_template(template)
# 创建ConversationBufferMemory
memory = ConversationBufferMemory()
# 初始化链
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
# 提问
res1 = chain.invoke({"question": "我的名字叫Tom"})
print(res1)
res1 = chain.invoke({"question": "你还记得我的名字吗"})
print(res1)

{'question': '我的名字叫Tom', 'history': '', 'text': '你好！很高兴见到你，Tom！如果你有任何问题或需要帮助的地方，请随时告诉我。我在这里为你提供帮助。😊'}
{'question': '你还记得我的名字吗', 'history': 'Human: 我的名字叫Tom\nAI: 你好！很高兴见到你，Tom！如果你有任何问题或需要帮助的地方，请随时告诉我。我在这里为你提供帮助。😊', 'text': '当然记得！我是AI助手，随时为你提供帮助。😊'}


举例2：可以通过memory_key修改memory数据的变量名

In [8]:
from langchain_classic.chains.llm import LLMChain
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

# 初始化大模型
llm = ChatOllama(model='deepseek-r1:7b', base_url="http://localhost:11434")
# 创建提示
# 有两个输入键：实际输入与来自记忆类的输入 需确保PromptTemplate和ConversationBufferMemory中的键匹配
template = """你可以与人类对话。
当前对话历史: {chat_his}
人类问题: {question}
回复:
"""
prompt = PromptTemplate.from_template(template)
# 创建ConversationBufferMemory(修改默认的memory key)
memory = ConversationBufferMemory(memory_key="chat_his")
# 初始化链
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
# 提问
res1 = chain.invoke({"question": "我的名字叫Tom"})
print(res1)
res1 = chain.invoke({"question": "我的名字是什么？"})
print(res1)

{'question': '我的名字叫Tom', 'chat_his': '', 'text': '你好，Tom！很高兴认识你。有什么我可以帮你的吗？'}
{'question': '我的名字是什么？', 'chat_his': 'Human: 我的名字叫Tom\nAI: 你好，Tom！很高兴认识你。有什么我可以帮你的吗？', 'text': '很高兴认识你，Tom！你的名字是Tom。有什么我可以帮你的吗？'}


举例3：使用ChatPromptTemplate 和 return_messages

In [9]:
# 1.导入相关包
from langchain_classic.chains.llm import LLMChain
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_ollama import ChatOllama

# 2.创建LLM
llm = ChatOllama(model='deepseek-r1:7b', base_url="http://localhost:11434")
# 3.创建Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name='history'),
    ("human", "问题：{question}")
])
# 4.创建Memory
memory = ConversationBufferMemory(return_messages=True)
# 5.创建LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)
# 6.调用LLMChain
res1 = llm_chain.invoke({"question": "中国首都在哪里？"})
print(res1, end="\n\n")
res2 = llm_chain.invoke({"question": "我刚刚问了什么"})
print(res2)

{'question': '中国首都在哪里？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国首都北京位于中国东部，具体位于 deletes the Tibet region, Gansu province, and Shanghai市。北京不仅是中国的政治、经济和文化中心，也是世界上最大的国际大都市之一。', additional_kwargs={}, response_metadata={})], 'text': '中国首都北京位于中国东部，具体位于 deletes the Tibet region, Gansu province, and Shanghai市。北京不仅是中国的政治、经济和文化中心，也是世界上最大的国际大都市之一。'}

{'question': '我刚刚问了什么', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国首都北京位于中国东部，具体位于 deletes the Tibet region, Gansu province, and Shanghai市。北京不仅是中国的政治、经济和文化中心，也是世界上最大的国际大都市之一。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么', additional_kwargs={}, response_metadata={}), AIMessage(content='您上一次问了：“问题：我刚刚问了什么”。', additional_kwargs={}, response_metadata={})], 'text': '您上一次问了：“问题：我刚刚问了什么”。'}


总结：通过上面的例子，我们可以得到，对于memory，PromptTemplate 和 ChatPromptTemplate有以下几种区别：
|特性|PromptTemplate|ChatPromptTemplate|
|----|----|----|
|消息存储时机|调用完成后|调用前存储用户输入，调用后存储模型输出|
|消息调用显示|第一次调用history变量为空|第一次history变量就已经存储完整对话信息|
|消息类型|字符串|List[BaseMessage]|

注意：
我们观察到的现象不是 bug，而是 LangChain 为 保障对话一致性 所做的刻意设计：
1. 用户提问后，系统应立即"记住"该问题
2. AI回答后，该响应应即刻加入对话上下文
3. 返回给客户端的结果应反映最新状态

# 4、ConversationChain
`ConversationChain`实质上就是封装了`LLMChain`和`ConversationBufferMemory`的Chain，提供了默认的PromptTemplate和Memory，你也可以不用。


举例1：使用自定义的PromptTemplate，省略Memory

In [12]:
from langchain_classic.chains.conversation.base import ConversationChain
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

# 初始化大模型
llm = ChatOllama(model='deepseek-r1:7b', base_url="http://localhost:11434")
# 创建提示
template = """你可以与人类对话。
当前对话历史: {history}
人类问题: {input}
回复:
"""
prompt = PromptTemplate.from_template(template)
# 初始化链
chain = ConversationChain(llm=llm, prompt=prompt)
# 提问
res1 = chain.invoke({"input": "我的名字叫Tom"})
print(res1)
res1 = chain.invoke({"input": "我的名字是什么？"})
print(res1)

{'input': '我的名字叫Tom', 'history': '', 'response': '你好！我是R，很高兴认识你。有什么我可以帮助你的吗？'}
{'input': '我的名字是什么？', 'history': 'Human: 我的名字叫Tom\nAI: 你好！我是R，很高兴认识你。有什么我可以帮助你的吗？', 'response': '你好！我叫Tom。很高兴认识你！有什么我可以帮助你的吗？'}


举例2：使用内置的提示词模板和Memory

In [15]:
from langchain_classic.chains.conversation.base import ConversationChain
from langchain_ollama import ChatOllama

# 初始化大模型
llm = ChatOllama(model='deepseek-r1:7b', base_url="http://localhost:11434")
# 初始化链
chain = ConversationChain(llm=llm)
# 提问
res1 = chain.invoke({"input": "我的名字叫Tom"})
print(res1)
res1 = chain.invoke({"input": "请问我叫什么"})
print(res1)

{'input': '我的名字叫Tom', 'history': '', 'response': 'Based on the conversation between Tom and the AI, here is an organized summary of the thought process:\n\n1. **Understanding the Context**: The user provided their name as "Tom" in English, prompting the AI for additional details about his full name.\n\n2. **Common Names Consideration**: Since Tom is a common first name, the AI considered possible surnames and middle names associated with it.\n\n3. **Possibilities Explored**:\n   - Tom could be a full name.\n   - Tom might have a middle name or an Asian surname.\n   - There\'s also the possibility of changing his name to another.\n\n4. **Conclusion**: Without further information, the best approach is to request more details from Tom to provide a precise response.\n\nThis structured thought process leads to the conclusion that additional information is needed before providing a specific answer.'}
{'input': '请问我叫什么', 'history': 'Human: 我的名字叫Tom\nAI: Based on the conversation between Tom a

# 5、ConversationBufferWindowMemory
通过上面的学习，我们知道`ConversationBufferMemory` 可以无限制地按照原始顺序存储完整的对话历史。但这样的缺点是，如果对话轮次过多，可能会导致内存溢出。
并且上下文信息过多，消耗的token也变得更多。此外，每个大模型也都有自己的token窗口限制.

我们发现，过久远的对话数据往往并不能对当前轮次的问答提供有效的信息，LangChain 给出的解决方式是： `ConversationBufferWindowMemory` 模块。该记忆类会保存一段时间内对话交互的列表， 仅使用最近 K 个交互 。这样就使缓存区不会变得太大。


**特点：**
* 适合长对话场景
* 与 Chains/Models 无缝集成
* 支持两种返回格式（通过 return_messages 参数控制输出格式）
    * return_messages=True 返回消息对象列表（ List[BaseMessage]）
    * return_messages=False （默认） 返回拼接的 纯文本字符串

## 场景1：入门使用
举例1：设置窗口存储阈值为1（只保存最近一条消息）

In [1]:
# 1.导入相关包
from langchain_classic.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手"})
memory.save_context({"input": "你的生日是哪天？"}, {"output": "我不清楚"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': 'Human: 你的生日是哪天？\nAI: 我不清楚'}


C:\Users\Administrator\AppData\Local\Temp\ipykernel_33848\35182604.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


举例2：`ConversationBufferWindowMemory` 也支持使用聊天模型（Chat Model）的情况，同样可以通过`return_messages=True` 参数，将对话转化为消息列表形式。

In [2]:
# 1.导入相关包
from langchain_classic.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=2, return_messages=True)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='初次对话，你能介绍一下你自己吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以了。我是一个无所不能的小智。', additional_kwargs={}, response_metadata={})]}


## 场景2：结合chain
借助提示词模版去构建LangChain

In [6]:
from langchain_classic.memory import ConversationBufferWindowMemory
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain_classic.chains.llm import LLMChain
from langchain_ollama import ChatOllama

# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
{history}
Human: {question}
AI:"""
# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)
# 4.创建大模型
llm = ChatOllama(model="deepseek-r1:7b", base_url="http://localhost:11434")
# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)
# 6.定义LLMChain
conversation_with_summary = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    verbose=True,
    )
# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question":"你好，我是孙小空"})
# print(respon1)
# 8.执行链（第二次提问）
respon2 =conversation_with_summary.invoke({"question":"我还有两个师弟，一个是猪小戒，一个是沙小僧"})
# print(respon2)
# 9.执行链（第三次提问）
respon3 =conversation_with_summary.invoke({"question":"我今年高考，竟然考上了1本"})
# print(respon3)
# 10.执行链（第四次提问）
respon4 =conversation_with_summary.invoke({"question":"我叫什么？"})
print(respon4)



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：

Human: 你好，我是孙小空
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 你好，我是孙小空
AI: 你好！很高兴见到你。请问有什么我可以为你效劳的吗？
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI: 好的！很高兴见到你。请问有什么我可以为你效劳的吗？
Human: 我今年高考，竟然考上了1本
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 我今年高考，竟然考上了1本
AI: AI: 听说你今年高考考上了1本，真是 congratulations！那是什么样的经历呢？有什么特别的故事或者感受吗？
Human: 我叫什么？
AI:

> Finished chain.
{'question': '我叫什么？', 'history': 'Human: 我今年高考，竟然考上了1本\nAI: AI: 听说你今年高考考上了1本，真是 congratulations！那是什么样